# Rapport de projet de session - IFT 712
### Objectif du projet : tester six méthodes de classification sur une base de données Kaggle.

Nous avons choisi la base de données "Heart Failure Prediction Dataset" puisqu'elle permet de faire de la classification sur un jeu de données réel et avec des applications concrètes.
Les méthodes de classification que nous allons tester sont les suivantes :
* Modèle génératif
* K plus proches voisins
* Régression logistique
* Méthode à noyaux
* SVM
* Perceptron multicouches

Pour cela, nous utiliserons la bibliothèque scikit-learn pour implémenter les algotihmes ainsi que pandas  pour traiter les données

Nous utiliserons également [Trello][1] ainsi que discord afin d'organiser le projet
Le code est versionné sur [Github][1] en suivant les conventions suivantes :
* conventionals [commits][3]
* merge requests sur master
* une branche par feature (i.e par algorithme de classification)

Le code et les commentaires sont rédigés en francais et suivant la convention [pep8][4]. Nous utiliserons la fonctionnalité "code with me" de pycharm permettant à plussieurs membres du groupe de coder sur le même projet en même temps

[1]: https://trello.com/b/U21MHLaj/projet-ift712-deadline-11-12-23
[2]: https://github.com/MorganChabaudENSSAT/projet_ift712
[3]: https://www.conventionalcommits.org/en/v1.0.0/
[4]: https://peps.python.org/pep-0008/

In [16]:
'''
 Imporation des bibliothèques python générales
'''
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

'''
 Imporation des bibliothèques spécifiques au devoir
'''

import utils

In [17]:
# Importation des données
df = pd.read_csv('heart.csv') # Dataframe contenant les données
columns_names = df.columns

# Visualisation des données pour mieux les comprendre
print(df.head())
print(df.dtypes)

   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR   
0   40   M           ATA        140          289          0     Normal    172  \
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
1              N      1.0     Flat             1  
2              N      0.0       Up             0  
3              Y      1.5     Flat             1  
4              N      0.0       Up             0  
Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64

In [18]:
# De cette visualisation des données, on  remarque que certaines features ne sont pas numériques ce qui empêche de les utiliser telles quelles dans les algorithmes de classification.
# => On va donc devoir traiter ces valeurs en les encodant.
le = LabelEncoder()

data = df.copy(deep = True)

data['Sex'] = le.fit_transform(data['Sex'])
data['ChestPainType'] = le.fit_transform(data['ChestPainType'])
data['RestingECG'] = le.fit_transform(data['RestingECG'])
data['ExerciseAngina'] = le.fit_transform(data['ExerciseAngina'])
data['ST_Slope'] = le.fit_transform(data['ST_Slope'])

# Les données sont mainteant toutes numériques et utilisables par les algorithmes de classification que nous mettrons en place
print(data)

     Age  Sex  ChestPainType  RestingBP  Cholesterol  FastingBS  RestingECG   
0     40    1              1        140          289          0           1  \
1     49    0              2        160          180          0           1   
2     37    1              1        130          283          0           2   
3     48    0              0        138          214          0           1   
4     54    1              2        150          195          0           1   
..   ...  ...            ...        ...          ...        ...         ...   
913   45    1              3        110          264          0           1   
914   68    1              0        144          193          1           1   
915   57    1              0        130          131          0           1   
916   57    0              1        130          236          0           0   
917   38    1              2        138          175          0           1   

     MaxHR  ExerciseAngina  Oldpeak  ST_Slope  Hear

## Classification par Machine à vecteurs de support (SVM)
Dans le cadre de ce projet, la base de classification à étudier est une celle d'une classification binaire. En effet, on chercher à savoir si le patient est atteint d'une pathologie cardiaque ou non. Dans ce cas, on s'intéresse à la prédiction de la colonne "HeartDisease", qui peut être soit vraie, soit fausse (soit 1, soit 0). On peut donc utiliser une classification à base de machine à vecteurs de support.

In [19]:
class SupportVectorMachine(object):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def train_svm(self):
        # Divisez les données en ensembles d'entraînement et de test
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.Y, test_size=0.2, random_state=42)

        # Définissez la grille des hyperparamètres à rechercher
        param_grid = {
            'C': [0.1, 1, 10, 100],  # Paramètre de régularisation
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Type de noyau
            'gamma': ['scale', 'auto', 0.1, 1, 10]  # Coefficient du noyau (pour les noyaux rbf, poly, et sigmoid)
        }

        # Créez un modèle SVM
        svm_model = SVC()

        # Utilisez GridSearchCV pour rechercher les meilleurs hyperparamètres
        grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
        grid_search.fit(X_train, y_train)

        # Affichez les meilleurs hyperparamètres
        print("Meilleurs hyperparamètres :", grid_search.best_params_)

        # Utilisez le modèle avec les meilleurs hyperparamètres pour faire des prédictions
        best_svm_model = grid_search.best_estimator_
        y_pred = best_svm_model.predict(X_test)
        print(classification_report(y_test, y_pred))

# Créez une instance de la classe SupportVectorMachine et entraînez le SVM avec optimisation des hyperparamètres
svm_instance = SupportVectorMachine(X=data.drop('HeartDisease', axis=1), Y=data['HeartDisease'])
svm_instance.train_svm()